In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import math
import numpy as np

def estimate_shoulder_height(rel_height, pitcher_height, lean_factor=0.85):
    """Estimate standing shoulder height as ~82% of pitcher height, adjusted for lean."""
    standing_shoulder_height = pitcher_height * 0.82
    shoulder_height = standing_shoulder_height * lean_factor
    return shoulder_height

def calculate_arm_angle(rel_height, rel_side, shoulder_height, shoulder_side=0.5):
    """Calculate arm angle in degrees using RelHeight, RelSide, and estimated shoulder height."""
    delta_height = rel_height - shoulder_height
    delta_side = rel_side - shoulder_side
    return math.degrees(math.atan2(delta_height, delta_side))

def break_plot(df: pd.DataFrame, ax: plt.Axes, pitcher_name: str, pitching_side: str, pitcher_height: float = 6.2):
    # Rename columns for consistency
    df = df.rename(columns={
        'HorzBreak': 'pfx_x',
        'InducedVertBreak': 'pfx_z',
        'TaggedPitchType': 'pitch_type',
        'PitcherThrows': 'p_throws'
    })

    # Pitch type mapping
    pitch_mapping = {
        'Fastball': '4-SEAM FASTBALL',
        'Sinker': 'SINKER',
        'Curveball': 'CURVEBALL',
        'Changeup': 'CHANGEUP',
        'Sweeper': 'SWEEPER',
        'Slider': 'SLIDER',
        'Cutter': 'CUTTER',
        'Splitter': 'SPLITTER',
        'FourSeamFastBall': '4-SEAM FASTBALL',
        'Undefined': 'UNKNOWN'
    }
    df['pitch_type'] = df['pitch_type'].fillna('UNKNOWN').astype(str).map(pitch_mapping).fillna('UNKNOWN')

    # Color dictionary
    dict_colour = {
        '4-SEAM FASTBALL': 'pink',
        'SINKER': 'purple',
        'CURVEBALL': 'blue',
        'CHANGEUP': 'orange',
        'SWEEPER': 'red',
        'SLIDER': 'green',
        'CUTTER': 'yellow',
        'SPLITTER': 'black',
        'FOURSEAM': 'pink',
        'UNKNOWN': 'gray'
    }
    missing_pitches = set(df['pitch_type']) - set(dict_colour.keys())
    if missing_pitches:
        raise ValueError(f"The palette dictionary is missing keys: {missing_pitches}")

    # Calculate shoulder height
    shoulder_height = estimate_shoulder_height(df['RelHeight'].mean(), pitcher_height)

    # Calculate arm angle for each pitch
    df['arm_angle'] = df.apply(lambda row: calculate_arm_angle(row['RelHeight'], row['RelSide'], shoulder_height), axis=1)

    # Filter based on pitcher throwing side and quadrant
    if pitching_side == 'R':
        # Right-handed: (+,+) quadrant (delta_height > 0, delta_side > 0)
        df = df[(df['RelHeight'] - shoulder_height > 0) & (df['RelSide'] - 0.5 > 0)]
    elif pitching_side == 'L':
        # Left-handed: (-,+) quadrant (delta_height > 0, delta_side < 0)
        df = df[(df['RelHeight'] - shoulder_height > 0) & (df['RelSide'] - 0.5 < 0)]
    else:
        raise ValueError("Pitcher throwing side is unknown or invalid.")

    # Calculate average arm angle for the y=x line
    avg_arm_angle = df['arm_angle'].mean()

    # Font properties
    font_properties = {'fontsize': 10}
    font_properties_axes = {'fontsize': 12, 'weight': 'bold'}
    font_properties_titles = {'fontsize': 14, 'weight': 'bold'}

    # Scatter plot of HorzBreak vs InducedVertBreak, sized by arm angle
    sns.scatterplot(ax=ax,
                    x=df['pfx_x'],
                    y=df['pfx_z'],
                    hue=df['pitch_type'],
                    size=df['arm_angle'],
                    sizes=(20, 200),  # Scale point sizes based on arm angle
                    palette=dict_colour,
                    ec='black',
                    alpha=0.8,
                    zorder=2)

    # Add y=x arm angle line in the appropriate quadrant
    if pitching_side == 'R':
        # (+,+) quadrant: y=x line from (0,0) to (25,25)
        x_vals = np.array([0, 25])
        y_vals = x_vals  # y=x
        ax.plot(x_vals, y_vals, color='black', linestyle='--', alpha=0.5, zorder=1)
        # Annotate average arm angle
        ax.text(20, 22, f'Arm Angle: {avg_arm_angle:.1f}°', fontstyle='italic', ha='right', va='top',
                bbox=dict(facecolor='white', edgecolor='black'), fontsize=10, zorder=3)
    elif pitching_side == 'L':
        # (-,+) quadrant: y=x line from (0,0) to (-25,25)
        x_vals = np.array([0, -25])
        y_vals = np.abs(x_vals)  # y=|x| to stay in (-,+) quadrant
        ax.plot(x_vals, y_vals, color='black', linestyle='--', alpha=0.5, zorder=1)
        # Annotate average arm angle
        ax.text(-20, 22, f'Arm Angle: {avg_arm_angle:.1f}°', fontstyle='italic', ha='left', va='top',
                bbox=dict(facecolor='white', edgecolor='black'), fontsize=10, zorder=3)

    # Add quadrant lines
    ax.axhline(y=0, color='#808080', alpha=0.5, linestyle='--', zorder=1)
    ax.axvline(x=0, color='#808080', alpha=0.5, linestyle='--', zorder=1)

    # Set labels and title
    ax.set_xlabel('Horizontal Break (in, Pitcher\'s View)', fontdict=font_properties_axes)
    ax.set_ylabel('Induced Vertical Break (in)', fontdict=font_properties_axes)
    ax.set_title(f"Pitch Breaks with Arm Angle\n{pitcher_name} - {pitching_side}", fontdict=font_properties_titles)

    # Set ticks and limits
    ax.set_xticks(range(-20, 21, 10))
    ax.set_xticklabels(range(-20, 21, 10), fontdict=font_properties)
    ax.set_yticks(range(-20, 21, 10))
    ax.set_yticklabels(range(-20, 21, 10), fontdict=font_properties)
    ax.set_xlim(-25, 25)
    ax.set_ylim(-25, 25)

    # Add quadrant annotations
    ax.text(-24.2, -24.2, s='← First Base', fontstyle='italic', ha='left', va='bottom',
            bbox=dict(facecolor='white', edgecolor='black'), fontsize=10, zorder=3)
    ax.text(24.2, -24.2, s='Third Base →', fontstyle='italic', ha='right', va='bottom',
            bbox=dict(facecolor='white', edgecolor='black'), fontsize=10, zorder=3)

    # Ensure equal aspect ratio
    ax.set_aspect('equal', adjustable='box')

    # Format ticks as integers
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))

    # Remove legend (optional, uncomment to keep it)
    ax.get_legend().remove()

# Example usage
df = pd.read_csv('Data/BeltersBees5-29.csv')
df = df[df['Pitcher'] == 'Alec Bergman']
pitcher_height = 6.2  # Example pitcher height
fig, ax = plt.subplots(figsize=(8, 8))
break_plot(df, ax, "Alec Bergman", "R", pitcher_height)
plt.show()